In [1]:
%run boilerplate.ipynb

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.18.1

numpy     : 1.19.2
matplotlib: 3.3.2
pyNN      : 0.9.5
neo       : 0.8.0
moviepy   : 1.0.3
imageio   : 2.9.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.4.0-56-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit

Hostname: inv-ope-de06

Git hash: f34be83854ac8bf458f81ef778714973cb6e6ef3

Git repo: https://github.com/SpikeAI/2020-12_brainhack_Project7

Git branch: main



In [2]:
input_movie = generate_dot( grid_width, grid_width,
                            time_bins,
                            flash_start=0.2,
                            flash_duration=0.6,
                            dot_size = 0.1,
                            im_noise = .1,
                            im_contrast = .9)
minmax(input_movie)
input_movie.shape

movie.min()=-0.47846708277294026 - movie.max()=1.0 


(64, 64, 350)

In [3]:
show(make_movie(input_movie, label='input_movie'))

TODO: use elephant surrogate poisson generator

In [4]:
input_movie_spikes = (input_movie > np.random.rand(grid_width, grid_width, time_bins))*1.
input_movie_spikes[..., :2] = 0 # avoid having a spike at the first time bin

In [5]:
minmax(input_movie), minmax(input_movie_spikes)

movie.min()=-0.47846708277294026 - movie.max()=1.0 
movie.min()=0.0 - movie.max()=1.0 


(None, None)

In [6]:
show(make_movie(input_movie_spikes, label='poisson'))

TODO: use a neo ImageSequence object

In [7]:
cellSourceSpikes = []
for i in range(N_pop):
    row = i//grid_width
    col = i%grid_width
    spike_times = np.where(input_movie_spikes[row, col, :]==True)[0]
    cellSourceSpikes.append(spike_times)

cellSourceSpikes = [list(elem) for elem in cellSourceSpikes]
len(cellSourceSpikes), cellSourceSpikes[0], np.sum([len(st) for st in cellSourceSpikes])

(4096, [36, 72, 122, 169, 175, 185, 269, 272, 283, 310, 321], 67802)

In [8]:
try:
    import pyNN.spiNNaker as sim
    simulator = 'spinnaker'
except ModuleNotFoundError:
    import pyNN.nest as sim
    simulator = 'nest'

CSAConnector: libneurosim support not available in NEST.
Falling back on PyNN's default CSAConnector.
Please re-compile NEST using --with-libneurosim=PATH


In [9]:
sim.setup(timestep=.1,
          min_delay=1,
          max_delay=100,
          debug=0)

sources = sim.SpikeSourceArray(spike_times=cellSourceSpikes)
spikeSource = sim.Population(N_pop, sources)    
spikeSource.record(['spikes'])

In [10]:
sim.run(simtime=simtime)

350.0

In [11]:
spikeSources  = spikeSource.get_data()#.segments[0].spiketrains
S_spikes = spikeSources.segments[0].spiketrains

In [12]:
sim.end()

In [13]:
st = S_spikes[0]
st

SpikeTrain
annotations: {'source_population': 'population0',
  'source_id': 4099,
  'source_index': 0}

In [14]:
st.as_array()

array([ 36.,  72., 122., 169., 175., 185., 269., 272., 283., 310., 321.])

In [15]:
st.magnitude 

array([ 36.,  72., 122., 169., 175., 185., 269., 272., 283., 310., 321.])

In [16]:
all_spikes = np.hstack([st.magnitude for st in S_spikes])
len(all_spikes)

67802

In [17]:
def spikes2numpy(spikelist):
    output_spike = np.zeros((grid_width, grid_width, time_bins))
    for i in range(N_pop):
        row = i//grid_width
        col = i%grid_width
        for spiketime in spikelist[i].as_array():
            output_spike[row, col, int(spiketime)] = 1
    return output_spike
    
output_spike = spikes2numpy(S_spikes)
minmax(output_spike)
output_spike[0, 0, :]

movie.min()=0.0 - movie.max()=1.0 


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

In [18]:
output_spike.shape

(64, 64, 350)

In [19]:
show(make_movie(output_spike, label='output_spike'))